In [3]:
import parse_genet
import os
import sys
import getopt

import vi

import importlib

import simulate
import torch
import matplotlib.pyplot as plt
import numpy as np

import pyro.distributions as dist

import scipy.stats

import pandas as pd
import numpy as np

In [4]:
param_dict = {
    'ref_dir' : '/gpfs/commons/groups/knowles_lab/data/ADSP_reguloML/LD_PRScs/ldblk_1kg_eur', ## add my path
    'bim_prefix' : "test_data/test",
    'sst_file' : "test_data/sumstats.txt",
    'n_gwas' : 200000, 
    'out_dir' : "test_data",
    "seed" : 42, 
    "beta_std" : "False", 
    "n_iter" : 1000
}
chrom=22



ref_df = parse_genet.parse_ref(param_dict['ref_dir'] + '/snpinfo_1kg_hm3')


ref_df = ref_df[ref_df.CHR == chrom]
vld_df = parse_genet.parse_bim(param_dict['bim_prefix'] + ".bim")
vld_df = vld_df[vld_df.CHR == chrom]
sst_dict = parse_genet.parse_sumstats(ref_df, vld_df, param_dict['sst_file'], param_dict['n_gwas']) ## take the interception SNP of sumstat, ref_ld, validate df
ld_blk, ld_blk_sym, blk_size = parse_genet.parse_ldblk(param_dict['ref_dir'], sst_dict, chrom)



... parse reference file: /gpfs/commons/groups/knowles_lab/data/ADSP_reguloML/LD_PRScs/ldblk_1kg_eur/snpinfo_1kg_hm3 ...
... parse sumstats file: test_data/sumstats.txt ...
... 1000 common SNPs in the reference, sumstats, and validation set ...
... parse reference LD on chromosome 22 ...


In [14]:
len(ld_blk) ## 4 blocks
len(ld_blk_sym) ## same 4 blocks
blk_size

[135, 176, 511, 178]

In [15]:
ld_blk[1]

array([[ 1.        , -0.594883  , -0.854701  , ..., -0.029778  ,
        -0.0611675 , -0.0623354 ],
       [-0.594883  ,  1.        ,  0.432201  , ...,  0.0220252 ,
         0.0248158 ,  0.028129  ],
       [-0.854701  ,  0.432201  ,  1.        , ..., -0.00325899,
         0.0440021 ,  0.0483626 ],
       ...,
       [-0.029778  ,  0.0220252 , -0.00325899, ...,  1.        ,
         0.88211   ,  0.876589  ],
       [-0.0611675 ,  0.0248158 ,  0.0440021 , ...,  0.88211   ,
         1.        ,  0.993641  ],
       [-0.0623354 ,  0.028129  ,  0.0483626 , ...,  0.876589  ,
         0.993641  ,  1.        ]])

In [25]:
np.array_equal(ld_blk_sym[0] , ld_blk[0])

True

In [28]:
#import h5py
hdf_chr = h5py.File(param_dict['ref_dir'] + '/ldblk_1kg_chr' + str(chrom) + '.hdf5', 'r')

In [32]:
n_blk = len(hdf_chr)

In [35]:
import scipy as sp
ld_blk = [sp.array(hdf_chr['blk_'+str(blk)]['ldblk']) for blk in range(1,n_blk+1)]

/gpfs/commons/home/tlin/.conda/envs/polyfun/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: scipy.array is deprecated and will be removed in SciPy 2.0.0, use numpy.array instead
  


In [38]:
snp_blk = []
for blk in range(1,n_blk+1):
    snp_blk.append([bb.decode("UTF-8") for bb in list(hdf_chr['blk_'+str(blk)]['snplist'])])

In [45]:
len(snp_blk)

24

In [46]:
### the way to read s
for i in list(hdf_chr['blk_4']['snplist']):
    print(i.decode("UTF-8"))
    break

rs17210001


In [51]:
    blk_size = []
    ld_blk_sym = []
    ld_blk_filt = []
    mm = 0
    for blk in range(n_blk):
        idx = [ii for (ii, snp) in enumerate(snp_blk[blk]) if snp in sst_dict['SNP'].to_numpy() ]
        print(len(idx))
        if len(idx) == 0: 
            continue
        
        blk_size.append(len(idx))
        
        idx_blk = np.arange(mm,mm+len(idx))
        flip = sst_dict['FLP'][idx_blk]
        ld_blk_here = ld_blk[blk][sp.ix_(idx,idx)]*sp.outer(flip,flip)
        ld_blk_filt.append(ld_blk_here)
        
        #_, s, v = linalg.svd(ld_blk_here)
        #h = sp.dot(v.T, sp.dot(sp.diag(s), v)) # just weird way of getting transpose?! 
        #ld_blk_sym.append( (ld_blk_here+h)/2 )
        
        ld_blk_sym.append( (ld_blk_here+ld_blk_here.T)/2 )

        mm += len(idx)

135
176
511
178
0
0

/gpfs/commons/home/tlin/.conda/envs/polyfun/lib/python3.6/site-packages/ipykernel_launcher.py:15: DeprecationWarning: scipy.ix_ is deprecated and will be removed in SciPy 2.0.0, use numpy.ix_ instead
  from ipykernel import kernelapp as app
/gpfs/commons/home/tlin/.conda/envs/polyfun/lib/python3.6/site-packages/ipykernel_launcher.py:15: DeprecationWarning: scipy.outer is deprecated and will be removed in SciPy 2.0.0, use numpy.outer instead
  from ipykernel import kernelapp as app



0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


In [56]:
blk_size

[135, 176, 511, 178]

In [55]:
mm

1000

## test_out simulate.py  

In [3]:
ref_df = ref_df[ref_df.CHR == chrom]
vld_df = parse_genet.parse_bim(param_dict['bim_prefix'] + ".bim")
vld_df = vld_df[vld_df.CHR == chrom]
sst_dict = parse_genet.parse_sumstats(ref_df, vld_df, param_dict['sst_file'], param_dict['n_gwas']) ## take the interception SNP of sumstat, ref_ld, validate df
ld_blk, ld_blk_sym, blk_size = parse_genet.parse_ldblk(param_dict['ref_dir'], sst_dict, chrom) ## ld_blk & ld_blk_sym are the equal here.


#mcmc_gtb.mcmc(param_dict['a'], param_dict['b'], param_dict['phi'], sst_dict, param_dict['n_gwas'], ld_blk, blk_size, param_dict['n_iter'], param_dict['n_burnin'], param_dict['thin'], int(chrom), param_dict['out_dir'], param_dict['beta_std'], param_dict['seed'])

beta_true, beta_mrg, annotations = simulate.simulate_sumstats(ld_blk, blk_size, param_dict['n_gwas'], p = len(sst_dict))

... parse sumstats file: test_data/sumstats.txt ...
... 1000 common SNPs in the reference, sumstats, and validation set ...
... parse reference LD on chromosome 22 ...


NameError: name 'p' is not defined

In [61]:
mm

1000

In [63]:
len(sst_dict)

1000

In [4]:
p = len(sst_dict)
nz = torch.rand(p) < 0.2
beta_true = torch.where(nz, 0.1 * torch.randn(p), torch.zeros(p))
beta_true

tensor([ 0.1412,  0.0000,  0.0969,  0.0030,  0.0000,  0.0000,  0.0000,  0.0000,
        -0.1608,  0.0000, -0.0199,  0.0743,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000, -0.1591,
         0.0000,  0.0000,  0.0000,  0.0916,  0.0637,  0.1197,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000, -0.0187, -0.0308,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000, -0.0488,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000, -0.0892,  0.0000,  0.0000,  0.0000, -0.0880,
         0.0000,  0.0592,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.1746, -0.0926,  0.0000,  0.1239,  0.0000,  0.0000,
         0.0000,  0.2192, -0.0286,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.1847,  0.0000,  0.0000,  0.0000,  0.0000,  0.1021,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.00

In [5]:
annotations = torch.stack([torch.ones(p),nz,torch.randn(p)])
annotations ## for every snp

tensor([[ 1.0000,  1.0000,  1.0000,  ...,  1.0000,  1.0000,  1.0000],
        [ 1.0000,  0.0000,  1.0000,  ...,  1.0000,  0.0000,  0.0000],
        [-0.3444,  0.8976,  0.0366,  ..., -0.7617,  0.5392, -0.6843]])

In [6]:
len(annotations[1])

1000

In [7]:
mm=0
for kk in range(len(ld_blk)):
    idx_blk = torch.arange(mm,mm+blk_size[kk])
    ld_torch = torch.tensor(ld_blk[kk], dtype = torch.float)
    print(idx_blk)
    print(ld_torch)
    L, V = torch.linalg.eigh(ld_torch)
    break

tensor([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
         14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,  27,
         28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,  41,
         42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,  53,  54,  55,
         56,  57,  58,  59,  60,  61,  62,  63,  64,  65,  66,  67,  68,  69,
         70,  71,  72,  73,  74,  75,  76,  77,  78,  79,  80,  81,  82,  83,
         84,  85,  86,  87,  88,  89,  90,  91,  92,  93,  94,  95,  96,  97,
         98,  99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111,
        112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125,
        126, 127, 128, 129, 130, 131, 132, 133, 134])
tensor([[ 1.0000, -0.1463,  0.2104,  ..., -0.0104, -0.0318,  0.0257],
        [-0.1463,  1.0000, -0.0522,  ..., -0.0410, -0.0468, -0.0328],
        [ 0.2104, -0.0522,  1.0000,  ..., -0.0468, -0.0579,  0.0332],
        ...,
        [-0.0104, -0.

In [16]:
L

tensor([-2.8701e-07, -2.5933e-07, -2.4489e-07, -1.9719e-07, -1.4229e-07,
        -9.1758e-08, -6.8323e-08, -4.0874e-08, -2.9425e-08, -1.5823e-08,
         1.0808e-09,  6.1763e-09,  2.8092e-08,  6.1921e-08,  8.2929e-08,
         1.2159e-07,  1.5997e-07,  2.2286e-07,  2.4746e-07,  3.6694e-07,
         7.9440e-07,  1.5274e-03,  1.6464e-03,  1.7237e-03,  1.7317e-03,
         2.0195e-03,  2.1671e-03,  2.3259e-03,  2.6552e-03,  2.7704e-03,
         3.1001e-03,  4.0371e-03,  4.2736e-03,  4.7468e-03,  5.0005e-03,
         5.4366e-03,  5.7589e-03,  6.5098e-03,  7.2803e-03,  7.5573e-03,
         7.8616e-03,  8.9578e-03,  9.3030e-03,  1.1276e-02,  1.1653e-02,
         1.2443e-02,  1.3063e-02,  1.3592e-02,  1.4437e-02,  1.7321e-02,
         1.7821e-02,  2.0109e-02,  2.1693e-02,  2.3077e-02,  2.4719e-02,
         2.7001e-02,  3.5196e-02,  3.6366e-02,  3.9364e-02,  3.9530e-02,
         4.2396e-02,  4.5874e-02,  5.1111e-02,  5.3921e-02,  5.6995e-02,
         5.8743e-02,  6.2307e-02,  6.7301e-02,  7.3

In [17]:
L[L < 0.] = 0. 

In [18]:
L

tensor([0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 1.0808e-09, 6.1763e-09,
        2.8092e-08, 6.1921e-08, 8.2929e-08, 1.2159e-07, 1.5997e-07, 2.2286e-07,
        2.4746e-07, 3.6694e-07, 7.9440e-07, 1.5274e-03, 1.6464e-03, 1.7237e-03,
        1.7317e-03, 2.0195e-03, 2.1671e-03, 2.3259e-03, 2.6552e-03, 2.7704e-03,
        3.1001e-03, 4.0371e-03, 4.2736e-03, 4.7468e-03, 5.0005e-03, 5.4366e-03,
        5.7589e-03, 6.5098e-03, 7.2803e-03, 7.5573e-03, 7.8616e-03, 8.9578e-03,
        9.3030e-03, 1.1276e-02, 1.1653e-02, 1.2443e-02, 1.3063e-02, 1.3592e-02,
        1.4437e-02, 1.7321e-02, 1.7821e-02, 2.0109e-02, 2.1693e-02, 2.3077e-02,
        2.4719e-02, 2.7001e-02, 3.5196e-02, 3.6366e-02, 3.9364e-02, 3.9530e-02,
        4.2396e-02, 4.5874e-02, 5.1111e-02, 5.3921e-02, 5.6995e-02, 5.8743e-02,
        6.2307e-02, 6.7301e-02, 7.3898e-02, 7.7166e-02, 8.3501e-02, 8.9109e-02,
        9.3467e-02, 1.0344e-01, 1.1249e-

In [94]:
ld_blk[0] ## ld_torch= ld_blk[0]

array([[ 1.       , -0.14631  ,  0.210427 , ..., -0.0103829, -0.0318179,
         0.0256598],
       [-0.14631  ,  1.       , -0.0522206, ..., -0.0409753, -0.0467828,
        -0.0328089],
       [ 0.210427 , -0.0522206,  1.       , ..., -0.0467965, -0.057888 ,
         0.0332181],
       ...,
       [-0.0103829, -0.0409753, -0.0467965, ...,  1.       ,  0.967532 ,
        -0.187081 ],
       [-0.0318179, -0.0467828, -0.057888 , ...,  0.967532 ,  1.       ,
        -0.214542 ],
       [ 0.0256598, -0.0328089,  0.0332181, ..., -0.187081 , -0.214542 ,
         1.       ]])

In [98]:
sigma_over_sqrt_n = 1. / torch.sqrt(torch.tensor(param_dict['n_gwas']))
beta_mrg = torch.zeros(p)
mm = 0
for kk in range(len(ld_blk)):
        idx_blk = torch.arange(mm,mm+blk_size[kk])
        ld_torch = torch.tensor(ld_blk[kk], dtype = torch.float)
        L, V = torch.linalg.eigh(ld_torch)
        L[L < 0.] = 0.

        beta_mrg[idx_blk] = ld_torch @ beta_true[idx_blk] + sigma_over_sqrt_n * (V @ torch.diag(L.sqrt())) @ torch.randn(blk_size[kk])
        #ld_torch @ beta_true[idx_blk], 
        # covariance_matrix = ld_torch * sigma_over_sqrt_n**2).rsample()
        mm += blk_size[kk]

In [100]:
annotations

tensor([[ 1.0000,  1.0000,  1.0000,  ...,  1.0000,  1.0000,  1.0000],
        [ 0.0000,  0.0000,  1.0000,  ...,  1.0000,  0.0000,  1.0000],
        [-0.1161, -1.0200, -1.1958,  ...,  0.2512,  1.5513,  0.2354]])

In [19]:
!python PRScs.py

^C
Traceback (most recent call last):
  File "PRScs.py", line 78, in <module>
    import vi
  File "/gpfs/commons/home/tlin/script/casopr/vi.py", line 1, in <module>
    import torch
  File "/gpfs/commons/home/tlin/.conda/envs/polyfun/lib/python3.6/site-packages/torch/__init__.py", line 704, in <module>
    from torch import multiprocessing as multiprocessing
  File "/gpfs/commons/home/tlin/.conda/envs/polyfun/lib/python3.6/site-packages/torch/multiprocessing/__init__.py", line 18, in <module>
    from .reductions import init_reductions
  File "/gpfs/commons/home/tlin/.conda/envs/polyfun/lib/python3.6/site-packages/torch/multiprocessing/reductions.py", line 15, in <module>
    import multiprocessing.resource_sharer
  File "<frozen importlib._bootstrap>", line 971, in _find_and_load
  File "<frozen importlib._bootstrap>", line 955, in _find_and_load_unlocked
  File "<frozen importlib._bootstrap>", line 665, in _load_unlocked
  File "<frozen importlib._bootstrap_external>", line 674, in 

In [14]:
#import pyro
pyro.sample("test",dist.Normal(0,1))

tensor(-0.6424)

In [13]:
dist.Normal(0,1).to_event(1)

ValueError: Expected reinterpreted_batch_ndims <= len(base_distribution.batch_shape), actual 1 vs 0

In [ ]:
 def load_annotations_file(self, args, chr_num, use_ridge):
        
        #load annotations file for this chromosome
        if use_ridge:
            annot_filenames = get_file_name(args, 'annot', chr_num, allow_multiple=True)
        else:
            annot_filenames = [get_file_name(args, 'bins', chr_num)]
        
        #load annotation file(s)
        df_annot_chr_list = []
        for annot_filename in annot_filenames:
            try:
                df_annot_chr = pd.read_parquet(annot_filename)
            except (ArrowIOError, ArrowInvalid):
                df_annot_chr = pd.read_table(annot_filename)
            df_annot_chr_list.append(df_annot_chr)
        if len(df_annot_chr_list)==1:
            df_annot_chr = df_annot_chr_list[0]
        else:
            for df in df_annot_chr_list[1:]:
                for snp_col in SNP_COLUMNS:
                    if (df.shape[0] != df_annot_chr_list[0].shape[0]) or (np.any(df[snp_col] != df_annot_chr_list[0][snp_col])):
                        raise ValueError('Different annotation files of chromosome %d must be perfectly aligned'%(chr_num))
                df.drop(columns=['CM'], inplace=True, errors='ignore')
                df.drop(columns=SNP_COLUMNS, inplace=True, errors='raise')
            df_annot_chr = pd.concat(df_annot_chr_list, axis=1)
        
        #make sure all required columns were found
        df_annot_chr.drop(columns=['CM'], inplace=True, errors='ignore')
        found_missing_col = False
        for colname in SNP_COLUMNS:
            if colname not in df_annot_chr.columns:
                logging.error('%s has a missing column: %s'%(annot_filename, colname))
                found_missing_col = True
        if found_missing_col:
            raise ValueError('Missing columns found in %s'%(annot_filename))
            
        #subset annotations if requested
        if args.anno is not None:
            anno_to_use = args.anno.split(',')
            assert np.all(np.isin(anno_to_use, df_annot_chr.columns))
            df_annot_chr = df_annot_chr[SNP_COLUMNS + anno_to_use]
            
        #if we have more annotations that ref-ld, it might mean that some annotations were removed, so remove them from here as well
        if not np.all(np.isin(self.ref_ld_cnames, df_annot_chr.columns)):
            missing_annot = [c for c in self.ref_ld_cnames if c not in df_annot_chr.columns]
            raise ValueError('The following annotations have LD-scores but are not in any of the annotation files: %s'%(missing_annot))
        if len(self.ref_ld_cnames) < len(df_annot_chr.columns) - len(SNP_COLUMNS):            
            df_annot_chr = df_annot_chr[SNP_COLUMNS + self.ref_ld_cnames]

        #make sure that we get the same columns as the ones in the LD-score files
        if not np.all([c for c in df_annot_chr.columns if c not in SNP_COLUMNS ]== self.ref_ld_cnames):
            raise ValueError('Annotation names in annotations file do not match the one in the LD-scores file')            

        return df_annot_chr